# 🎓 Module 1: The "Why" of Reducers
**Objective:** Understand why standard Python lists overwrite data in LangGraph and how Reducers fix this.

### Prerequisites
Ensure you have the necessary libraries installed.

In [1]:
%pip install langgraph langchain-openai langchain-core

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


---

### Flavor 1: The Broken State (Default Behavior)
In a standard `TypedDict`, updating a key replaces the old value. Watch how the user's input is lost.

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

# 1. Define State (No Reducer)
class BrokenState(TypedDict):
    messages: list[str] 

# 2. Define Node
def chatbot_node(state: BrokenState):
    # This return statement REPLACES the 'messages' key
    return {"messages": ["Bot: I am a goldfish. I forget everything."]}

# 3. Build Graph
builder = StateGraph(BrokenState)
builder.add_node("chatbot", chatbot_node)


builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)


broken_graph = builder.compile()

# 4. Execute
print("--- RUN 1 ---")
res = broken_graph.invoke({"messages": ["User: Hi"]})
print(f"Result: {res['messages']}") 
# FAIL: The user's 'Hi' is gone.

--- RUN 1 ---
Result: ['Bot: I am a goldfish. I forget everything.']


---

### Flavor 2: The Manual Fix
We can fix this by manually fetching the old list and appending to it. This works, but it is error-prone.

In [2]:
# 1. State
class ManualState(TypedDict):
    messages: list 

# 2. Node (Manual Appending Logic)
def chatbot_node(state: ManualState):
    # A. FETCH existing
    existing = state['messages']
    # B. CREATE new
    new_msg = "Bot: I remember, but I had to do the work manually!"
    # C. COMBINE manually
    return {"messages": existing + [new_msg]}

# 3. Build Graph
builder = StateGraph(ManualState)
builder.add_node("chatbot", chatbot_node)


builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)


manual_graph = builder.compile()

# 4. Execute
print("--- RUN 1 ---")
res = manual_graph.invoke({"messages": ["User: Hi"]})
print(f"Result: {res['messages']}")

--- RUN 1 ---
Result: ['User: Hi', 'Bot: I remember, but I had to do the work manually!']


---

### Flavor 3: The LangGraph Way (Reducers)
We use `Annotated` and `add_messages`. This tells LangGraph to handle the appending automatically. This is the best practice.

In [3]:
from typing import Annotated
from langgraph.graph.message import add_messages 
from langchain_core.messages import HumanMessage, AIMessage

# 1. State (Annotated with Reducer)
class SmartState(TypedDict):
    # "When you get a new message, ADD it to the list. Don't replace."
    messages: Annotated[list, add_messages] 

# 2. Node (Pure Logic)
def chatbot_node(state: SmartState):
    # We just return the NEW item. LangGraph handles the rest.
    return {"messages": [AIMessage(content="Bot: I remember everything automatically!")]}

# 3. Graph
builder = StateGraph(SmartState)

builder.add_node("chatbot", chatbot_node)

builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

smart_graph = builder.compile()

# 4. Execute
print("--- RUN 1 ---")
res = smart_graph.invoke({"messages": [HumanMessage(content="User: Hi")]})
print(f"Result: {res['messages']}")

print("\n--- RUN 2 ---")
# Simulate continuing conversation
current_history = res['messages']
res_2 = smart_graph.invoke({"messages": current_history + [HumanMessage(content="User: Bye")]})
print(f"Final History: {res_2['messages']}")

--- RUN 1 ---
Result: [HumanMessage(content='User: Hi', additional_kwargs={}, response_metadata={}, id='eff15446-689c-4670-a84d-3321b2064b16'), AIMessage(content='Bot: I remember everything automatically!', additional_kwargs={}, response_metadata={}, id='9695955d-54f0-4a0b-a1fe-938c3bcb8c92')]

--- RUN 2 ---
Final History: [HumanMessage(content='User: Hi', additional_kwargs={}, response_metadata={}, id='eff15446-689c-4670-a84d-3321b2064b16'), AIMessage(content='Bot: I remember everything automatically!', additional_kwargs={}, response_metadata={}, id='9695955d-54f0-4a0b-a1fe-938c3bcb8c92'), HumanMessage(content='User: Bye', additional_kwargs={}, response_metadata={}, id='08e51114-52fc-4aa1-aedd-bccb235ef10b'), AIMessage(content='Bot: I remember everything automatically!', additional_kwargs={}, response_metadata={}, id='bce5dd4b-37f2-40af-af2f-3cf19638894d')]
